In [1]:
from utils import random_walk, generate_walks, read_train_val_graph, sparse_mx_to_torch_sparse_tensor
from utils import apply_word2vec_on_features, create_and_normalize_adjacency, save_subgraph_in_file

In [8]:
from random import random
from random import randint
from datetime import datetime
import torch

In [3]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
import pandas as pd
import numpy as np
from time import time
from gensim.models import Word2Vec
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [10]:
save_subgraph_in_file(nbr_nodes=1000)

1000 nodes, 2405 edges Graph extracted from edgelist.txt
999
980 nodes, 2405 edges Graph saved in small_edgelist.txt
999


In [109]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def read_train_val_graph(path='../input_data/edgelist.txt', val_ratio=0.1):
    G = nx.read_edgelist(path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    nodes = list(G.nodes())
    n = G.number_of_nodes()
    m = G.number_of_edges()
    edges = list(G.edges())

    print('Number of nodes:', n, 'number of edges:', m,'in All the set')

    node_to_idx = dict()
    for i, node in enumerate(nodes):
        node_to_idx[node] = i

    val_edges = list()
    G_train = G.copy()

    for edge in edges:
        if random() < val_ratio and edge[0] < n and edge[1] < n:
            val_edges.append(edge)
            G_train.remove_edge(edge[0], edge[1]) # We remove the val edges from the graph G

   
    #for edge in val_edges:
        

    n = G_train.number_of_nodes()
    m = G_train.number_of_edges()
    train_edges = list(G_train.edges())

    print('Number of nodes:', n, 'number of edges:', m, 'in the Training set')
    print('len(nodes)', len(nodes))

    y_val = [1]*len(val_edges)

    n_val_edges = len(val_edges)
    
    print('Creating random val_edges...')
    for i in range(n_val_edges):
        n1 = nodes[randint(0, n-1)]
        n2 = nodes[randint(0, n-1)]
        (n1, n2) = (min(n1, n2), max(n1, n2))
        while n2 > n: #or (n1, n2) in train_edges:
            if (n1, n2) in train_edges:
                print((n1, n2), 'in train_edges:')
            else: print((n1, n2))
            n1 = nodes[randint(0, n-1)]
            n2 = nodes[randint(0, n-1)]
            (n1, n2) = (min(n1, n2), max(n1, n2))
        val_edges.append((n1, n2))
    
#     print('Cleaning random val_edges...')
#     l = len(intersection(val_edges,train_edges))
#     print('len(intersection(val_edges, train_edges))', len(intersection(val_edges, train_edges)))
#     for i, val_edge in enumerate(intersection(val_edges, train_edges)):
#         print(i+1, '/', l)
#         n1 = nodes[randint(0, n-1)]
#         n2 = nodes[randint(0, n-1)]
#         (n1, n2) = (min(n1, n2), max(n1, n2))
#         while (n1, n2) in train_edges or (n1, n2) in val_edges or n2 > n:
#             n1 = nodes[randint(0, n-1)]
#             n2 = nodes[randint(0, n-1)]
#             (n1, n2) = (min(n1, n2), max(n1, n2))
#         val_edges = [(n1, n2) if item == val_edge else item for item in val_edges]

# another way to clean::
# for edge in list(set(val_edges) & set(train_edges)):
#     val_edges.remove(edge)


    y_val.extend([0]*(n_val_edges))
    
    print('Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects')
    print('Loaded from', path[path.rfind('/')+1:], 'and with a training validation split ratio =', val_ratio)
    
    return G, G_train, train_edges, val_edges, y_val, nodes, node_to_idx

In [110]:
path = '../input_data/edgelist.txt'

G, G_train, train_edges, val_edges, y_val, nodes, node_to_idx = read_train_val_graph(val_ratio=0.1, path=path)

Number of nodes: 138499 number of edges: 1091955 in All the set
Number of nodes: 138499 number of edges: 983250 in the Training set
len(nodes) 138499
Creating random val_edges...
Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects
Loaded from edgelist.txt and with a training validation split ratio = 0.1


In [111]:
G.number_of_edges(), G_train.number_of_edges()

(1091955, 983250)

In [117]:
counter = 0
for val_edge in val_edges:
    if val_edge in G.edges():
        counter += 1
print(counter)

counter = 0
for val_edge in val_edges:
    if val_edge in G_train.edges():
        counter += 1
print(counter)

108725
18


In [116]:

walks = generate_walks(G=G_train, num_walks=10, walk_length=15)
features_np = apply_word2vec_on_features(features=walks, nodes=nodes)
adj, indices = create_and_normalize_adjacency(G_train)

Start generating walks....
Random walks generated in in 64s!
Start applying Word2Vec...
Word2vec model trained on features in 3 min!
(138499, 128) features numpy array created in 3 min!


/Users/ghassenabdedayem/Documents/Data/Polytechnique/5- Data Challenge/data_challenge_2022/draft_nb/utils.py:147: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph


Created a normalized adjancency matrix of shape (138499, 138499)
Created indices (2, 2104999) with the positions of non zeros in adj matrix


In [114]:
G = nx.read_edgelist('../input_data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
print(G.number_of_edges)
print(G_train.edges(2))
print(G.edges(2))
print(element in G_train.edges(2))
print(element in G.edges(2))
for element in G.edges(2):
    if element not in G_train.edges(2):
        print(element)


<bound method Graph.number_of_edges of <networkx.classes.graph.Graph object at 0x7f91b8167fd0>>
[(2, 0), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (2, 32), (2, 34), (2, 35), (2, 36), (2, 37), (2, 38), (2, 40), (2, 41), (2, 42), (2, 43), (2, 44), (2, 46), (2, 48), (2, 49), (2, 50), (2, 51), (2, 53), (2, 54), (2, 55), (2, 56), (2, 57), (2, 58), (2, 59), (2, 60), (2, 62), (2, 63)]
[(2, 0), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (2, 31), (2, 32), (2, 33), (2, 34), (2, 35), (2, 36), (2, 37), (2, 38), (2, 39), (2, 40), (2, 41), (2, 42), (2, 43), (2, 44), (2, 45), (2, 46), (2, 48), (2, 49), (2, 50), (2, 51), (2, 52), (2, 53), (2, 54), (2, 55), (2, 56), (2, 57), (2, 58), (2, 59), (2, 60), (2, 61), (2, 62), (2, 63)]
False
True
(2, 31)
(2, 33)
(2, 39)
(2, 45)
(2, 52)
(2, 61)


In [118]:
val_edges

[(1, 10),
 (2, 31),
 (2, 33),
 (2, 39),
 (2, 45),
 (2, 52),
 (2, 61),
 (5, 71762),
 (7, 55111),
 (7, 55118),
 (7, 55335),
 (10, 111357),
 (10, 105975),
 (13, 95564),
 (15, 121201),
 (15, 120800),
 (15, 118059),
 (16, 54943),
 (16, 101847),
 (16, 101814),
 (16, 81307),
 (16, 101855),
 (17, 103529),
 (17, 105330),
 (17, 105026),
 (19, 101822),
 (20, 111127),
 (20, 101829),
 (20, 86130),
 (20, 123441),
 (22, 7287),
 (22, 44193),
 (22, 64198),
 (22, 49125),
 (23, 120871),
 (23, 107491),
 (25, 58),
 (26, 49831),
 (31, 721),
 (32, 407),
 (37, 96455),
 (38, 729),
 (39, 43),
 (39, 115087),
 (40, 407),
 (40, 59304),
 (42, 127462),
 (43, 429),
 (43, 58),
 (44, 407),
 (48, 107414),
 (48, 14104),
 (52, 113584),
 (53, 96317),
 (53, 58),
 (54, 112488),
 (56, 105474),
 (62, 698),
 (48051, 16970),
 (48051, 16999),
 (48051, 28054),
 (48051, 38936),
 (48051, 39016),
 (48051, 107793),
 (48051, 66342),
 (48051, 54140),
 (101494, 101487),
 (111705, 124645),
 (107826, 119178),
 (111339, 123649),
 (110133, 9

In [121]:
G_train.edges(1), G.edges(1)

(EdgeDataView([(1, 0), (1, 3), (1, 5), (1, 6), (1, 7), (1, 9), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24)]),
 EdgeDataView([(1, 0), (1, 3), (1, 5), (1, 6), (1, 7), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24)]))

In [122]:
# Create class labels
y = np.zeros(2*indices.shape[1])
y[:indices.shape[1]] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.


In [123]:
# Transforms the numpy matrices/vectors to torch tensors.
features = torch.FloatTensor(features_np).to(device)
y = torch.LongTensor(y).to(device)
if type(adj) != torch.Tensor:
    adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
indices = torch.LongTensor(indices).to(device)

In [124]:
class GNN(nn.Module):
    def __init__(self, n_feat, n_hidden, n_class, dropout):
        super(GNN, self).__init__()
        self.fc1 = nn.Linear(n_feat, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_hidden)
        self.double_fc3 = nn.Linear((2*n_hidden), n_hidden)
        self.fc4 = nn.Linear(n_hidden, n_class)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        
        

    def forward(self, x_in, adj, pairs):
        
        h1 = self.fc1(x_in)
        z1 = self.relu(torch.mm(adj, h1))
        z1 = self.dropout(z1)

        h2 = self.fc2(z1)
        z2 = self.relu(torch.mm(adj, h2))

        #x = z2[pairs[0,:],:] - z2[pairs[1,:],:] # embedded features (z2) of node 0 - embedded features of node 1
        x1 = z2[pairs[0]]
        x2 = z2[pairs[1]]
        # could we add a new dimension to pairs to specify if same author(s)? and then what could we do?
              
        x = torch.cat((x1, x2), dim=1)        
        x = self.relu(self.double_fc3(x))        
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return F.log_softmax(x, dim=1)

In [128]:
epochs = 100
n_hidden = 128
dropout_rate = 0.2
n_class = 2
n_features = features.shape[1]

# Creates the model and specifies the optimizer
model = GNN(n_features, n_hidden, n_class, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [129]:
import time

# Train model
model.train()
start_time = time.time()
for epoch in range(epochs):
    t = time.time()
    optimizer.zero_grad()
    rand_indices = torch.randint(0, features.size(0), (indices.size(0),indices.size(1)), device=adj.device)# We take random indices each time we run an epoch
    pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices.   
    output = model(features, adj, pairs) # we run the model that gives the output.
    loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
    #print(type(loss_train), '\n', loss_train.shape)
    acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
    loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
    optimizer.step() # Performs a single optimization step (parameter update).
    
    if epoch % 5 == 0:
        print('Epoch: {:03d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'time: {:.4f} s'.format(time.time() - t),
             'total_time: {} min'.format(round((time.time() - start_time)/60)))

print("Optimization Finished in {} min!".format(round((time.time() - start_time)/60)))
print()



Epoch: 001 loss_train: 0.6931 acc_train: 0.5013 time: 34.0421 s total_time: 1 min
Epoch: 006 loss_train: 0.6638 acc_train: 0.6660 time: 25.2962 s total_time: 3 min
Epoch: 011 loss_train: 0.4971 acc_train: 0.7480 time: 22.6470 s total_time: 5 min
Epoch: 016 loss_train: 0.4738 acc_train: 0.7818 time: 22.8302 s total_time: 7 min
Epoch: 021 loss_train: 0.4411 acc_train: 0.8108 time: 24.8623 s total_time: 9 min
Epoch: 026 loss_train: 0.4052 acc_train: 0.8203 time: 25.3849 s total_time: 11 min
Epoch: 031 loss_train: 0.3833 acc_train: 0.8319 time: 22.7382 s total_time: 13 min
Epoch: 036 loss_train: 0.3653 acc_train: 0.8418 time: 23.5530 s total_time: 15 min
Epoch: 041 loss_train: 0.3532 acc_train: 0.8458 time: 25.5614 s total_time: 17 min
Epoch: 046 loss_train: 0.3411 acc_train: 0.8526 time: 22.1799 s total_time: 19 min
Epoch: 051 loss_train: 0.3325 acc_train: 0.8587 time: 25.8525 s total_time: 21 min
Epoch: 056 loss_train: 0.3204 acc_train: 0.8657 time: 25.7711 s total_time: 23 min
Epoch: 06

In [130]:
# Evaluating the model
model.eval()
eval_pairs = np.array(np.transpose(val_edges))
print(eval_pairs.shape)
eval_pairs = torch.LongTensor(eval_pairs).to(device)
print(eval_pairs.shape)
eval_output = model(features, adj, eval_pairs)
print(eval_output.shape)
y_pred = torch.exp(eval_output)
y_pred = y_pred.detach().cpu().numpy()

y_val_pred_true = list()
for element in y_pred:
    y_val_pred_true.append(element[1])
    
print('Log loss:', log_loss(y_val, y_pred_true))

(2, 217410)
torch.Size([2, 217410])
torch.Size([217410, 2])
Log loss: 2.430132547449547


In [135]:
y_pred_true

[0.0068791644,
 0.51461416,
 0.8669021,
 0.36618185,
 0.020769881,
 0.9832517,
 0.5173123,
 0.6228757,
 0.75390375,
 0.93110675,
 2.0570105e-05,
 0.98765576,
 0.92761695,
 0.00077021314,
 0.7263025,
 3.0105695e-05,
 0.986623,
 0.7775482,
 0.8152208,
 0.04375075,
 4.2025167e-05,
 0.086349435,
 0.94377106,
 0.73130643,
 0.9754456,
 0.0006405931,
 0.95496994,
 0.0011800342,
 2.1838985e-06,
 0.9563583,
 0.9677939,
 0.97538483,
 0.76104665,
 0.88995045,
 0.9866052,
 0.8217835,
 0.9142653,
 0.93872434,
 0.07839595,
 9.0892775e-07,
 0.9366415,
 0.98872584,
 4.6704403e-07,
 0.9268164,
 0.99513865,
 0.055306844,
 4.53322e-06,
 1.3047533e-06,
 0.91571563,
 0.98373574,
 0.27989337,
 0.82782567,
 0.01180967,
 0.6880035,
 0.007686667,
 0.0021729802,
 0.00015756862,
 0.076484,
 0.87629396,
 0.75086284,
 0.9829269,
 0.24153973,
 0.00027305738,
 0.7829623,
 0.00022818931,
 0.00013948919,
 0.17347898,
 0.10724894,
 5.4507407e-05,
 0.150281,
 0.9491255,
 0.7924724,
 0.94902766,
 0.003370582,
 0.96380657

In [132]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [133]:
from datetime import datetime
import pandas as pd



# Read test data. Each sample is a pair of nodes
node_pairs = list()
with open('../test_data/test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

# Testing
model.eval()
node_pairs = np.array(np.transpose(node_pairs))
pairs = torch.LongTensor(node_pairs).to(device)
output = model(features, adj, pairs)
y_pred = torch.exp(output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])
    

    
today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)

pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
"../submissions_files/{}-submission-{}.csv".format(today, random_nb), header=True, index=True, index_label='id'
)


In [ ]:
test_features = np.array([[5, 3, 0], [2, 1, 0], [4, 0, 1], [0, 3, 2]])
test_features = torch.LongTensor(test_features).to(device)
test_pairs = [[0, 0, 1, 2, 3, 2, 0, 1, 2, 3], [1, 2, 0, 0, 2, 3, 0, 1, 2, 3]]
test_pairs = torch.LongTensor(test_pairs).to(device)
print(test_features.shape, test_pairs.shape)
test_features[test_pairs[0]]

In [ ]:
y_pred.shape, np.shape(y_val), np.shape(y_pred_true)

In [ ]:
#print(output.shape)
print(features.shape, adj.shape, eval_pairs.shape)

In [ ]:
z2[pairs[1,:]]

In [ ]:
break

In [ ]:
break

In [ ]:
# Create class labels
y = np.zeros(4*G_train.number_of_edges())
y[:2*G_train.number_of_edges()] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.
train_edges[0]

In [ ]:
t = time()
X_train = np.zeros((4*m, 2*features_np.shape[1]))

for i, edge in enumerate(train_edges):
    X_train[i] = np.concatenate((features_np[train_edges[i][0]], features_np[train_edges[i][1]]), axis=0)
    X_train[m+i] = np.concatenate((features_np[train_edges[i][0]], features_np[train_edges[i][1]]), axis=0)
    X_train[2*m+i] = np.concatenate((features_np[randint(0,n-1)], features_np[randint(0,n-1)]), axis=0)
    X_train[3*m+i] = np.concatenate((features_np[randint(0,n-1)], features_np[randint(0,n-1)]), axis=0)
    
X_val = np.zeros((len(val_edges), 2*features_np.shape[1]))
for i, edge in enumerate(val_edges):
    X_val[i] = np.concatenate((features_np[val_edges[i][0]], features_np[val_edges[i][1]]), axis=0)
    
print('X_train and X_val created in {} s!'.format(round(time()-t)))

In [ ]:
t = time()
# Use logistic regression to predict if two nodes are linked by an edge
clf = LogisticRegression()
clf.fit(X_train, y)
y_pred = clf.predict_proba(X_val)
y_pred = y_pred[:,1]
print('Logistic regression performed in {} s!'.format(round(time()-t)))

In [ ]:
print('Log loss:', log_loss(y_val, y_pred))

In [ ]:
from datetime import datetime

# Read test data. Each sample is a pair of nodes
test_edges = list()
with open('../test_data/test.txt', 'r') as f:
    for line in f:
        t = line.split(',')
        test_edges.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

X_test = np.zeros((len(test_edges), 2*features_np.shape[1]))
for i, edge in enumerate(test_edges):
    X_test[i] = np.concatenate((features_np[test_edges[i][0]], features_np[test_edges[i][1]]), axis=0)

t = time()
# Use logistic regression to predict if two nodes are linked by an edge
y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]
print('Logistic regression performed in {} s!'.format(round(time()-t)))


today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 100000)

pd.DataFrame(y_pred, columns={'predicted'}).to_csv(
"../submissions_files/{}-submission-{}.csv".format(today, random_nb), header=True, index=True, index_label='id'
)
    
    
# # Testing
# node_pairs = np.array(np.transpose(node_pairs))
# pairs = torch.LongTensor(node_pairs).to(device)
# output = model(features, adj, pairs)
# y_pred = torch.exp(output)
# y_pred = y_pred.detach().cpu().numpy()

# y_pred_true = list()
# for element in y_pred:
#     y_pred_true.append(element[1])
    

    
# today = datetime.today().strftime('%Y-%m-%d')
# random_nb = randint(0, 100000)

# pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
# "../submissions_files/{}-submission-{}.csv".format(today, random_nb), header=True, index=True, index_label='id'
# )